In [1]:
import torch
import prdc
import numpy as np

# run only very first time

torch.hub.download_url_to_file("https://github.com/pytorch/vision/archive/v0.9.0.zip", "./vision-0.9.0.zip", hash_prefix=None, progress=True)

!unzip ./vision-0.9.0.zip

Downloading: "https://github.com/pytorch/vision/archive/v0.9.0.zip" to /root/.cache/torch/hub/v0.9.0.zip
Downloading: "https://download.pytorch.org/models/inception_v3_google-1a9a5a14.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-1a9a5a14.pth


  0%|          | 0.00/104M [00:00<?, ?B/s]

0.00B [00:00, ?B/s]

Archive:  ./vision-0.9.0.zip
01dfa8ea81972bb74b52dc01e6a1b43b26b62020
   creating: vision-0.9.0/
   creating: vision-0.9.0/.circleci/
 extracting: vision-0.9.0/.circleci/.gitignore  
   creating: vision-0.9.0/.circleci/build_docs/
  inflating: vision-0.9.0/.circleci/build_docs/commit_docs.sh  
  inflating: vision-0.9.0/.circleci/config.yml  
  inflating: vision-0.9.0/.circleci/config.yml.in  
  inflating: vision-0.9.0/.circleci/regenerate.py  
   creating: vision-0.9.0/.circleci/smoke_test/
   creating: vision-0.9.0/.circleci/smoke_test/docker/
  inflating: vision-0.9.0/.circleci/smoke_test/docker/Dockerfile  
   creating: vision-0.9.0/.circleci/unittest/
   creating: vision-0.9.0/.circleci/unittest/linux/
   creating: vision-0.9.0/.circleci/unittest/linux/scripts/
  inflating: vision-0.9.0/.circleci/unittest/linux/scripts/environment.yml  
  inflating: vision-0.9.0/.circleci/unittest/linux/scripts/install.sh  
  inflating: vision-0.9.0/.circleci/unittest/linux/scripts/post_process.sh 

In [2]:
import torch
import prdc
import numpy as np

inception_model = torch.hub.load('./vision-0.9.0', 'inception_v3', pretrained=True, source='local')

In [3]:
def _forward(self, x):
    
    x = self._transform_input(x)
    
    resize = torchvision.transforms.Resize((299,299))
    x = resize(x)
    
    
    # N x 3 x 299 x 299
    x = self.Conv2d_1a_3x3(x)
    # N x 32 x 149 x 149
    x = self.Conv2d_2a_3x3(x)
    # N x 32 x 147 x 147
    x = self.Conv2d_2b_3x3(x)
    # N x 64 x 147 x 147
    x = self.maxpool1(x)
    # N x 64 x 73 x 73
    x = self.Conv2d_3b_1x1(x)
    # N x 80 x 73 x 73
    x = self.Conv2d_4a_3x3(x)
    # N x 192 x 71 x 71
    x = self.maxpool2(x)
    # N x 192 x 35 x 35
    x = self.Mixed_5b(x)
    # N x 256 x 35 x 35
    x = self.Mixed_5c(x)
    # N x 288 x 35 x 35
    x = self.Mixed_5d(x)
    # N x 288 x 35 x 35
    x = self.Mixed_6a(x)
    # N x 768 x 17 x 17
    x = self.Mixed_6b(x)
    # N x 768 x 17 x 17
    x = self.Mixed_6c(x)
    # N x 768 x 17 x 17
    x = self.Mixed_6d(x)
    # N x 768 x 17 x 17
    x = self.Mixed_6e(x)
    # N x 768 x 17 x 17
    aux: Optional[Tensor] = None
    if self.AuxLogits is not None:
        if self.training:
            aux = self.AuxLogits(x)
    # N x 768 x 17 x 17
    x = self.Mixed_7a(x)
    # N x 1280 x 8 x 8
    x = self.Mixed_7b(x)
    # N x 2048 x 8 x 8
    x = self.Mixed_7c(x)
    # N x 2048 x 8 x 8
    # Adaptive average pooling
    x = self.avgpool(x)
    # N x 2048 x 1 x 1
    feature = x.detach()
    x = self.dropout(x)
    # N x 2048 x 1 x 1
    x = torch.flatten(x, 1)
    # N x 2048
    x = self.fc(x)
    # N x 1000 (num_classes)
    return x, feature, aux

inception_model.forward = _forward

In [13]:
from scipy.stats import entropy
def predict_to_inceptionscore(predict, splits=1) : 
    preds = torch.softmax(predict, dim=1).numpy()

    # Now compute the mean kl-div
    split_scores = []

    for k in range(splits):
        part = preds
        py = np.mean(part, axis=0)
        scores = []
        for i in range(part.shape[0]):
            pyx = part[i, :]
            scores.append(entropy(pyx, py))
        split_scores.append(np.exp(np.mean(scores)))
        
    return np.mean(split_scores), np.std(split_scores)

In [5]:
def feature_to_mu_sig(act):
    mu = np.mean(act, axis=0)
    sigma = np.cov(act, rowvar=False)
    return mu, sigma
    
    
from scipy import linalg
def calculate_frechet_distance(mu1, sigma1, mu2, sigma2, eps=1e-6):
    """Numpy implementation of the Frechet Distance.
    The Frechet distance between two multivariate Gaussians X_1 ~ N(mu_1, C_1)
    and X_2 ~ N(mu_2, C_2) is
            d^2 = ||mu_1 - mu_2||^2 + Tr(C_1 + C_2 - 2*sqrt(C_1*C_2)).
    """

    mu1 = np.atleast_1d(mu1)
    mu2 = np.atleast_1d(mu2)

    sigma1 = np.atleast_2d(sigma1)
    sigma2 = np.atleast_2d(sigma2)

    assert mu1.shape == mu2.shape, \
        'Training and test mean vectors have different lengths'
    assert sigma1.shape == sigma2.shape, \
        'Training and test covariances have different dimensions'

    diff = mu1 - mu2

    
    covmean, _ = linalg.sqrtm(sigma1.dot(sigma2), disp=False)
    if not np.isfinite(covmean).all():
        msg = ('fid calculation produces singular product; '
               'adding %s to diagonal of cov estimates') % eps
        print(msg)
        offset = np.eye(sigma1.shape[0]) * eps
        covmean = linalg.sqrtm((sigma1 + offset).dot(sigma2 + offset))

    
    if np.iscomplexobj(covmean):
        if not np.allclose(np.diagonal(covmean).imag, 0, atol=1e-3):
            m = np.max(np.abs(covmean.imag))
            raise ValueError('Imaginary component {}'.format(m))
        covmean = covmean.real

    tr_covmean = np.trace(covmean)

    return (diff.dot(diff) + np.trace(sigma1) +
            np.trace(sigma2) - 2 * tr_covmean)

In [6]:
import torchvision.transforms as transforms
import torchvision

dataset = torchvision.datasets.CIFAR10(root='../20210306_gan/dataset', #download=True,
                           transform=transforms.Compose([
                               transforms.Resize((64,64)),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))

data = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True, num_workers=4)
real_image, label = next(iter(data))
fake_image = torch.rand(real_image.shape)

In [9]:
inception_model.eval()
real_predict_notsoftmax, real_feature, _ = inception_model(inception_model, real_image)
fake_predict_notsoftmax, fake_feature, _ = inception_model(inception_model, fake_image)

In [14]:
real_inception_score = predict_to_inceptionscore(real_predict_notsoftmax.detach())
fake_inception_score = predict_to_inceptionscore(fake_predict_notsoftmax.detach())

In [15]:
real_feature_np = real_feature.detach().view(-1,2048).numpy()
fake_feature_np = fake_feature.detach().view(-1,2048).numpy()

real_mu, real_sigma = feature_to_mu_sig(real_feature_np)
fake_mu, fake_sigma = feature_to_mu_sig(fake_feature_np)

In [16]:
fid = calculate_frechet_distance(real_mu, real_sigma, fake_mu, fake_sigma)

In [17]:
real_pick = np.random.permutation(real_feature_np)[:10000]
fake_pick = np.random.permutation(fake_feature_np)[:10000]
metrics = prdc.compute_prdc(real_features=real_pick, fake_features=fake_pick, nearest_k=5)

metrics['fid'] = float(fid)
metrics['is_r'] = float(real_inception_score[0])
metrics['is_f'] = float(fake_inception_score[0])

print(metrics)

Num real: 32 Num fake: 32
{'precision': 0.0, 'recall': 0.0, 'density': 0.0, 'coverage': 0.0, 'fid': 491.02867097655866, 'is_r': 4.110505104064941, 'is_f': 1.021426796913147}
